In [18]:
from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl
from splink.duckdb.duckdb_comparison_library import (
    exact_match,
    levenshtein_at_thresholds,
    jaro_winkler_at_thresholds
)
import splink.duckdb.duckdb_comparison_template_library as ctl
import pandas as pd

In [19]:
from recordlinkage import datasets

dfA,dfB = datasets.load_febrl4()
dfA = dfA.reset_index()
dfB = dfB.reset_index()
dfA["cluster"] = dfA["rec_id"].apply(lambda x: "-".join(x.split('-')[:2]))
dfB["cluster"] = dfB["rec_id"].apply(lambda x: "-".join(x.split('-')[:2]))

In [20]:
dfA.to_excel('Febrl_Data4A.xlsx', index=False)
dfB.to_excel('Febrl_Data4B.xlsx', index=False)

In [21]:
dfA['unique_id'] = [x for x in range(1,len(dfA)+1)]
dfB['unique_id'] = [x for x in range(1,len(dfB)+1)]

In [22]:
dfs = [dfA,dfB]

In [23]:
from splink.duckdb.duckdb_linker import DuckDBLinker

basic_settings = {
    "link_type": "link_only",
    # NB as we are linking one-one, we know the probability that a random pair will be a match
    # hence we could set:
    # "probability_two_random_records_match": 1/5000,
    # however we will not specify this here, as we will use this as a check that
    # our estimation procedure returns something sensible
}

linker = DuckDBLinker(dfs, basic_settings)

In [24]:
cols_to_profile = list(dfs[0].columns)
cols_to_profile = [col for col in cols_to_profile if col not in ("rec_id", "cluster")]
linker.profile_columns(cols_to_profile)

In [25]:
blocking_rules = [
    "l.given_name = r.given_name AND l.surname = r.surname",
    "l.date_of_birth = r.date_of_birth",
    "l.soc_sec_id = r.soc_sec_id",
    "l.state = r.state AND l.address_1 = r.address_1",
    "l.street_number = r.street_number AND l.address_1 = r.address_1",
    "l.postcode = r.postcode",
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

In [26]:
import splink.duckdb.duckdb_comparison_level_library as cll
import splink.duckdb.duckdb_comparison_library as cl
import splink.duckdb.duckdb_comparison_template_library as ctl

# the simple model only considers a few columns, and only two comparison levels for each
simple_model_settings = {
    **basic_settings,
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        cl.exact_match("given_name", term_frequency_adjustments=True),
        cl.exact_match("surname", term_frequency_adjustments=True),
        cl.exact_match("street_number", term_frequency_adjustments=True),
    ],
    "retain_intermediate_calculation_columns": True,
}
# the detailed model considers more columns, using the information we saw in the exploratory phase
# we also include further comparison levels to account for typos and other differences
detailed_model_settings = {
    **basic_settings,
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        ctl.name_comparison("given_name", term_frequency_adjustments_name=True),
        ctl.name_comparison("surname", term_frequency_adjustments_name=True),
        cl.levenshtein_at_thresholds("date_of_birth", [1, 2]),
        cl.levenshtein_at_thresholds("soc_sec_id", [1, 2]),
        cl.exact_match("street_number", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("postcode", [1, 2], term_frequency_adjustments=True),
        # we don't consider further location columns as they will be strongly correlated with postcode
    ],
    "retain_intermediate_calculation_columns": True,
}


linker_simple = DuckDBLinker(dfs, simple_model_settings)
linker_detailed = DuckDBLinker(dfs, detailed_model_settings)

In [27]:
deterministic_rules = [
    "l.soc_sec_id = r.soc_sec_id",
    "l.given_name = r.given_name and l.surname = r.surname and l.date_of_birth = r.date_of_birth",
]

linker_detailed.estimate_probability_two_random_records_match(deterministic_rules, recall=0.8)

Probability two random records match is estimated to be  0.000238.
This means that amongst all possible pairwise record comparisons, one in 4,195.51 are expected to match.  With 25,000,000 total possible comparisons, we expect a total of around 5,958.75 matching pairs


In [28]:
linker_detailed.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - given_name (no m values are trained).
    - surname (no m values are trained).
    - date_of_birth (no m values are trained).
    - soc_sec_id (no m values are trained).
    - street_number (no m values are trained).
    - postcode (no m values are trained).


In [29]:
session_dob = linker_detailed.estimate_parameters_using_expectation_maximisation(
    "l.date_of_birth = r.date_of_birth"
)
session_pc = linker_detailed.estimate_parameters_using_expectation_maximisation(
    "l.postcode = r.postcode"
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.date_of_birth = r.date_of_birth

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - soc_sec_id
    - street_number
    - postcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth

Iteration 1: Largest change in params was 0.378 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.00439 in probability_two_random_records_match
Iteration 3: Largest change in params was 5.74e-05 in the m_probability of soc_sec_id, level `All other comparisons`

EM converged after 3 iterations

Your model is not yet fully trained. Missing estimates for:
    - date_of_birth (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.postcode = r.postcode

Parameter estima

In [30]:
predictions = linker_detailed.predict()
df_predictions = predictions.as_pandas_dataframe()
df_predictions.head(5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,given_name_l,given_name_r,gamma_given_name,tf_given_name_l,...,gamma_postcode,tf_postcode_l,tf_postcode_r,bf_postcode,bf_tf_adj_postcode,address_1_l,address_1_r,state_l,state_r,match_key
0,32.980182,1.000000,__splink__input_table_0,__splink__input_table_1,15,4841,matthew,matthew,3,0.007251,...,1,0.0002,0.0003,0.912841,1.000000,cowcher place,cowcher dlace,vic,vic,0
1,50.631350,1.000000,__splink__input_table_0,__splink__input_table_1,18,1557,jessica,jessica,3,0.008805,...,3,0.0003,0.0003,722.334657,3.880469,bishopbourne court,NaN,nsw,nsw,0
2,37.792422,1.000000,__splink__input_table_0,__splink__input_table_1,48,3511,harry,harry,3,0.004765,...,3,0.0013,0.0013,722.334657,0.895493,hodgson crescent,charteris crescent,nsw,nsw,0
3,33.522744,1.000000,__splink__input_table_0,__splink__input_table_1,80,3673,riley,riley,3,0.005904,...,3,0.0008,0.0008,722.334657,1.455176,clem hill street,clem hill street,vic,nsw,0
4,17.423825,0.999994,__splink__input_table_0,__splink__input_table_1,84,3440,benjamin,benjamin,3,0.010048,...,3,0.0006,0.0006,722.334657,1.940235,NaN,NaN,nsw,nsw,0


In [31]:
clusters = linker_detailed.cluster_pairwise_predictions_at_threshold(predictions, threshold_match_probability=0.99)
df_clusters = clusters.as_pandas_dataframe().sort_values("cluster_id")
df_clusters.groupby("cluster_id").size().value_counts()

Completed iteration 1, root rows count 0


2    4945
1     110
dtype: int64

In [34]:
df_clusters.to_excel('df_clusters.xlsx')

In [ ]:
linker.estimate_parameters_using_expectation_maximisation("l.date_of_birth = r.date_of_birth")
linker.estimate_parameters_using_expectation_maximisation("l.postcode = r.postcode")